In [1]:
import nibabel as nib

import numpy as np

from utils import read_tracking, write_tracking

In [2]:
tracking_path = "../lab_al.ts"
dest_path = "../lab_al_corrected.csv"

tracking = read_tracking(tracking_path)

In [3]:
# Create rotation matrix around x axis 4x4 transformation matrix
rotate_x = lambda theta: np.array([[1, 0, 0, 0],
                                      [0, np.cos(theta), -np.sin(theta), 0],
                                      [0, np.sin(theta), np.cos(theta), 0],
                                      [0, 0, 0, 1]])

# Create rotation matrix around y axis 4x4 transformation matrix
rotate_y = lambda theta: np.array([[np.cos(theta), 0, np.sin(theta), 0],
                                      [0, 1, 0, 0],
                                      [-np.sin(theta), 0, np.cos(theta), 0],
                                      [0, 0, 0, 1]])

# Create rotation matrix around z axis 4x4 transformation matrix
rotate_z = lambda theta: np.array([[np.cos(theta), -np.sin(theta), 0, 0],
                                      [np.sin(theta), np.cos(theta), 0, 0],
                                      [0, 0, 1, 0],
                                      [0, 0, 0, 1]])

# Create transformation matrix for flipping x axis
flip_x = np.array([[-1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

# Create transformation matrix for flipping y axis
flip_y = np.array([[1, 0, 0, 0],
                   [0, -1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

# Create transformation matrix for flipping z axis
flip_z = np.array([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])


In [4]:
# Flip in y axis
flip_y_axis = lambda x: np.dot(flip_y, x)
tracking = np.apply_along_axis(flip_y_axis, 1, tracking)

degrees = 40
rad = np.deg2rad(degrees)

# Rotate around y axis by 45 degrees
rotate_y_axis = lambda x: np.dot(rotate_y(-rad), x)
tracking = np.apply_along_axis(rotate_y_axis, 1, tracking)

In [5]:
# Create translation matrix
translation = np.eye(4, dtype=np.float32)
translation[1, 3] -= 537 / 2 * 0.172780

# Apply translation
tracking = np.apply_along_axis(lambda x: np.dot(translation, x), 1, tracking)

write_tracking(dest_path, tracking)

In [6]:
path = "../lab_al.nii.gz"
img = nib.load(path)
data = img.get_fdata()

print(data.shape)

(221, 537, 1714)


In [7]:
print(f"Tracking shape: {tracking.shape}")

Tracking shape: (1714, 4, 4)


In [8]:
# Create leg data

# Swap image axes
data = np.swapaxes(data, 1, 2)
data = np.swapaxes(data, 0, 1)
data = np.swapaxes(data, 1, 2)

In [9]:
import os
import shutil
from PIL import Image

In [10]:
print(f"Data shape: {data.shape}")

Data shape: (1714, 537, 221)


In [11]:
path = "leg_data"
if os.path.isdir(path):
    shutil.rmtree(path)
os.makedirs(path)

os.makedirs(os.path.join(path, "images"))

In [12]:
# Save images.npy
np.save(os.path.join(path, "images.npy"), data)
# Save poses.npy
np.save(os.path.join(path, "poses.npy"), tracking)

data = data.astype(np.uint8)

# Save images
for i in range(data.shape[0]):
    img = Image.fromarray(data[i, :, :])
    img.save(os.path.join(path, "images", f"{i}.png"))

array([[[ 33,  32,  32, ...,  32,  63, 178],
        [146, 140, 137, ..., 126, 120,  51],
        [137, 132, 126, ..., 104, 113,  52],
        ...,
        [ 25,  27,  25, ...,  23,  26,  12],
        [ 24,  24,  25, ...,  31,  29,  13],
        [ 28,  25,  21, ...,  27,  24,  10]],

       [[ 33,  31,  32, ...,  30,  62, 178],
        [146, 140, 137, ..., 125, 117,  48],
        [138, 132, 125, ..., 114, 122,  54],
        ...,
        [ 25,  27,  25, ...,  22,  32,  15],
        [ 24,  23,  25, ...,  19,  25,  15],
        [ 28,  27,  27, ...,  25,  25,  15]],

       [[ 33,  32,  32, ...,  28,  65, 181],
        [146, 140, 137, ..., 129, 133,  58],
        [137, 131, 125, ..., 128, 132,  53],
        ...,
        [ 30,  28,  21, ...,  29,  31,  14],
        [ 34,  31,  24, ...,  29,  29,  13],
        [ 30,  29,  28, ...,  27,  28,  13]],

       ...,

       [[ 33,  31,  33, ...,  31,  70, 182],
        [151, 142, 143, ..., 140, 169,  75],
        [142, 142, 137, ..., 139, 163,  69

In [27]:
path = "C:/Users/Bugra/Desktop/masters-thesis/data/LAB_AL_G.nii"
img = nib.load(path)
affine = img.affine

leg_img_path = "leg_data.nii.gz"
leg_conf_path = "leg_conf.nii.gz"

# Read leg image
leg_img = nib.load(leg_img_path)
leg_img_data = leg_img.get_fdata()

# Read leg config
leg_conf = nib.load(leg_conf_path)
leg_conf_data = leg_conf.get_fdata()

if np.max(leg_conf_data) == 1:
    leg_conf_data *= 255

# Save them with the same affine
leg_img = nib.Nifti1Image(leg_img_data, affine, header=img.header)
leg_conf = nib.Nifti1Image(leg_conf_data, affine, header=img.header)

nib.save(leg_img, leg_img_path)
nib.save(leg_conf, leg_conf_path)

In [29]:
path = "../liver_all_sweeps_confidence_maps.nii.gz"
img = nib.load(path)
data = img.get_fdata()

print(data)

[[[1.         1.         1.         ... 1.         1.         1.        ]
  [0.99997443 0.99997205 0.99997026 ... 0.98014516 0.98775399 0.99602377]
  [0.99994916 0.99994445 0.99994063 ... 0.96123046 0.97617388 0.99228954]
  ...
  [0.13419192 0.11529994 0.10220107 ... 0.31902823 0.37293169 0.47762594]
  [0.13160586 0.11299144 0.10005885 ... 0.31818327 0.37190199 0.47624221]
  [0.12895252 0.11058798 0.0978179  ... 0.31718788 0.37069339 0.47461766]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [0.99997419 0.99997193 0.99997032 ... 0.98049754 0.98861915 0.99630994]
  [0.9999488  0.99994421 0.99994105 ... 0.96192259 0.97739857 0.99300891]
  ...
  [0.13046663 0.11188418 0.0987448  ... 0.32534853 0.38137901 0.48903221]
  [0.12787905 0.109551   0.09660061 ... 0.32440242 0.38021526 0.48746493]
  [0.12528315 0.10718586 0.0943979  ... 0.32331103 0.37887985 0.48566958]]

 [[1.         1.         1.         ... 1.         1.         1.        ]
  [0.99997395 0.999971